In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
pd.set_option('display.max_rows', None)
%matplotlib inline
def disp(X):
    display(pd.DataFrame(X))


def call_election(total_pop=10000, blue_pct=60, seed=42):
    rnd = np.random.RandomState(seed)
    blue = {'clr':'blue', 'pct':blue_pct}
    red  = {'clr':'red',  'pct': 100 - blue_pct}
    d = 10
    num_districts = d**2

    fig, ax = plt.subplots()
    df = pd.DataFrame()
    dot_size = 1000*d**2 / total_pop**1.3
    for c in [blue, red]:
        c['pop'] = round(total_pop * c['pct'] / 100)
        c['loc'] = rnd.uniform(0, d, size=(2,c['pop']))
        ax.scatter(*c['loc'], s = dot_size, color=c['clr'])
        c['dist_pop'] = np.zeros([d, d], int)
        for i,j in zip(*np.floor(c['loc']).astype(int)):
            c['dist_pop'][i,j] += 1
        df[c['clr']] = c['dist_pop'].ravel()

    for x in np.arange(0,d+1):
        ax.plot([x,x],[0,d], color='black')
    for y in np.arange(0,d+1):
        ax.plot([0,d],[y,y], color='black')
    plt.show()
    # plt.close()

    df['pop'] = df['blue'] + df['red']
    D = df['blue'] - df['red']
    df['diff'] = D 
    df['diff pct'] = D / df['pop'] * 100
    df.loc[D > 0,'winner'] = 'blue'
    df.loc[D < 0,'winner'] = 'red'
    df.loc[D ==0,'winner'] = 'tie'
    df.sort_values('diff pct', inplace=True)

    win = pd.DataFrame(df['winner'].value_counts(normalize=True, sort=True).round(3)*100)
    win.columns = ['districts won']
    win['pop pct'] = df[['blue','red']].sum()/df['pop'].sum()*100
    disp(win)
    
#     disp(df['diff pct'].describe())
    print('Average difference = {:.1f}%'.format(df['diff pct'].mean()))
    disp(df.round(3))




total_pop_slider = widgets.IntSlider(min=100, max=100000, step=100, value=10000, description='Total Pop', continuous_update=False)
blue_pct_slider = widgets.IntSlider(min=0, max=100, value=50, description='Blue %', continuous_update=False)
seed_slider = widgets.IntSlider(min=1, max=100, value=64, description='Rand Seed', continuous_update=False)

img = widgets.interactive_output(call_election, {'total_pop':total_pop_slider, 'blue_pct':blue_pct_slider, 'seed':seed_slider})
widgets.VBox([total_pop_slider, blue_pct_slider, seed_slider, img])

VBox(children=(IntSlider(value=10000, continuous_update=False, description='Total Pop', max=100000, min=100, step=100), IntSlider(value=50, continuous_update=False, description='Blue %'), IntSlider(value=64, continuous_update=False, description='Rand Seed', min=1), Output()))